In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
from pprint import pprint

# Import API key
from api_keys import g_key

In [3]:
# read csv file and make it dataframe
covid_csv = "Resources/owid-covid-data.csv"
covid_df = pd.read_csv(covid_csv)
covid_df

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,excess_mortality
0,AFG,Asia,Afghanistan,2020-02-24,5.0,5.0,NaN,NaN,NaN,NaN,...,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN
1,AFG,Asia,Afghanistan,2020-02-25,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN
2,AFG,Asia,Afghanistan,2020-02-26,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN
3,AFG,Asia,Afghanistan,2020-02-27,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN
4,AFG,Asia,Afghanistan,2020-02-28,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116560,ZWE,Africa,Zimbabwe,2021-09-09,126056.0,125.0,134.000,4521.0,4.0,10.286,...,21.4,307.846,1.82,1.6,30.7,36.791,1.7,61.49,0.571,NaN
116561,ZWE,Africa,Zimbabwe,2021-09-10,126163.0,107.0,118.857,4532.0,11.0,10.714,...,21.4,307.846,1.82,1.6,30.7,36.791,1.7,61.49,0.571,NaN
116562,ZWE,Africa,Zimbabwe,2021-09-11,126220.0,57.0,113.571,4536.0,4.0,10.000,...,21.4,307.846,1.82,1.6,30.7,36.791,1.7,61.49,0.571,NaN
116563,ZWE,Africa,Zimbabwe,2021-09-12,126269.0,49.0,102.714,4538.0,2.0,8.000,...,21.4,307.846,1.82,1.6,30.7,36.791,1.7,61.49,0.571,NaN


In [4]:
# group the data set by location and find the max for the total cases up-to-date
group_covid = covid_df.groupby(["location"])["total_cases"].max()
group_covid

location
Afghanistan             154094.0
Africa                 8057274.0
Albania                 157436.0
Algeria                 200301.0
Andorra                  15096.0
                        ...     
Wallis and Futuna            NaN
World                225270139.0
Yemen                     8452.0
Zambia                  207960.0
Zimbabwe                126399.0
Name: total_cases, Length: 233, dtype: float64

In [9]:
# drop NAN values to clean the dataset
removena_covid = group_covid.dropna(inplace=False)
# sort the data in descending form
removena_covid = pd.DataFrame(removena_covid).sort_values(by="total_cases",ascending=False)
# remove any duplicated data 
removedulicate_covid = removena_covid.drop_duplicates()
# from descending sorting, can observe that the data contains the world data and continent data, remove these data
remove_Continents = ["World","Asia","Europe","North America","South America","European Union","Oceania"]
clean_covid = removedulicate_covid.drop(remove_Continents).reset_index()
# visualise the clean data
clean_covid["total_cases/pre million"] = clean_covid["total_cases"]/1000000
clean_covid

,location,total_cases,total_cases/pre million
0,United States,41221266.0,41.221266
1,India,33289579.0,33.289579
2,Brazil,21006424.0,21.006424
3,Africa,8057274.0,8.057274
4,United Kingdom,7290168.0,7.290168
...,...,...,...
188,Solomon Islands,20.0,0.000020
189,Vanuatu,4.0,0.000004
190,Samoa,3.0,0.000003
191,Kiribati,2.0,0.000002


In [10]:
clean_covid.describe()

,total_cases,total_cases/pre million
count,1.930000e+02,193.000000
mean,1.208946e+06,1.208946
std,4.258100e+06,4.258100
min,1.000000e+00,0.000001
25%,1.776700e+04,0.017767
50%,1.572610e+05,0.157261
75%,6.167650e+05,0.616765
max,4.122127e+07,41.221266


In [5]:
# Retriving lat and lng for locations
for index,row in clean_covid.iterrows():
    location = row["location"]
    # Build the endpoint URL
    target_url = f"https://maps.googleapis.com/maps/api/geocode/json?address={location}&key={g_key}"


    # Run a request to endpoint and convert result to json
    geo_data = requests.get(target_url).json()
    # Print the json
    pprint(geo_data)
    try:
        # Extract latitude and longitude for the locations in dataset
        lat = geo_data["results"][0]["geometry"]["location"]["lat"]
        clean_covid.loc[index,"Lat"] = lat
        lng = geo_data["results"][0]["geometry"]["location"]["lng"]
        clean_covid.loc[index,"Lng"] = lng
    except:
        print("Cannot find the location")

{'results': [{'address_components': [{'long_name': 'United States',
                                      'short_name': 'US',
                                      'types': ['country', 'political']}],
              'formatted_address': 'United States',
              'geometry': {'bounds': {'northeast': {'lat': 71.5388001,
                                                    'lng': -66.885417},
                                      'southwest': {'lat': 18.7763,
                                                    'lng': 170.5957}},
                           'location': {'lat': 37.09024, 'lng': -95.712891},
                           'location_type': 'APPROXIMATE',
                           'viewport': {'northeast': {'lat': 49.38,
                                                      'lng': -66.94},
                                        'southwest': {'lat': 25.82,
                                                      'lng': -124.39}}},
              'place_id': 'ChIJCzYy5IS16lQRQrfeQ5K

{'results': [{'address_components': [{'long_name': 'Iran',
                                      'short_name': 'IR',
                                      'types': ['country', 'political']}],
              'formatted_address': 'Iran',
              'geometry': {'bounds': {'northeast': {'lat': 39.782056,
                                                    'lng': 63.3333366},
                                      'southwest': {'lat': 24.8066999,
                                                    'lng': 44.0326949}},
                           'location': {'lat': 32.427908, 'lng': 53.688046},
                           'location_type': 'APPROXIMATE',
                           'viewport': {'northeast': {'lat': 39.782056,
                                                      'lng': 63.3333366},
                                        'southwest': {'lat': 24.8066999,
                                                      'lng': 44.0326949}}},
              'place_id': 'ChIJ8R1rwS7s9z4RzvpRn

{'results': [{'address_components': [{'long_name': 'Poland',
                                      'short_name': 'PL',
                                      'types': ['country', 'political']}],
              'formatted_address': 'Poland',
              'geometry': {'bounds': {'northeast': {'lat': 54.9054761,
                                                    'lng': 24.1458931},
                                      'southwest': {'lat': 49.002025,
                                                    'lng': 14.1228641}},
                           'location': {'lat': 51.919438, 'lng': 19.145136},
                           'location_type': 'APPROXIMATE',
                           'viewport': {'northeast': {'lat': 54.9054761,
                                                      'lng': 24.1458931},
                                        'southwest': {'lat': 49.002025,
                                                      'lng': 14.1228641}}},
              'place_id': 'ChIJuwtkpGSZAEcR6

{'results': [{'address_components': [{'long_name': 'Czechia',
                                      'short_name': 'CZ',
                                      'types': ['country', 'political']}],
              'formatted_address': 'Czechia',
              'geometry': {'bounds': {'northeast': {'lat': 51.0557185,
                                                    'lng': 18.8592361},
                                      'southwest': {'lat': 48.5518081,
                                                    'lng': 12.090589}},
                           'location': {'lat': 49.81749199999999,
                                        'lng': 15.472962},
                           'location_type': 'APPROXIMATE',
                           'viewport': {'northeast': {'lat': 51.0557185,
                                                      'lng': 18.8592361},
                                        'southwest': {'lat': 48.5518081,
                                                      'lng': 12.09058

{'results': [{'address_components': [{'long_name': 'Israel',
                                      'short_name': 'IL',
                                      'types': ['country', 'political']}],
              'formatted_address': 'Israel',
              'geometry': {'bounds': {'northeast': {'lat': 33.33280500000001,
                                                    'lng': 35.896244},
                                      'southwest': {'lat': 29.47969999999999,
                                                    'lng': 34.2673871}},
                           'location': {'lat': 31.046051, 'lng': 34.851612},
                           'location_type': 'APPROXIMATE',
                           'viewport': {'northeast': {'lat': 33.33280500000001,
                                                      'lng': 35.896244},
                                        'southwest': {'lat': 29.47969999999999,
                                                      'lng': 34.2673871}}},
              'p

...

{'results': [{'address_components': [{'long_name': 'Equatorial Guinea',
                                      'short_name': 'GQ',
                                      'types': ['country', 'political']}],
              'formatted_address': 'Equatorial Guinea',
              'geometry': {'bounds': {'northeast': {'lat': 3.8355,
                                                    'lng': 11.3333},
                                      'southwest': {'lat': -1.5475,
                                                    'lng': 5.541900099999999}},
                           'location': {'lat': 1.650801, 'lng': 10.267895},
                           'location_type': 'APPROXIMATE',
                           'viewport': {'northeast': {'lat': 3.8355,
                                                      'lng': 11.3333},
                                        'southwest': {'lat': -1.5475,
                                                      'lng': 5.541900099999999}}},
              'place_id': '

{'results': [{'address_components': [{'long_name': 'Niger',
                                      'short_name': 'NE',
                                      'types': ['country', 'political']}],
              'formatted_address': 'Niger',
              'geometry': {'bounds': {'northeast': {'lat': 23.4999997,
                                                    'lng': 15.9990339},
                                      'southwest': {'lat': 11.693756,
                                                    'lng': 0.1617177}},
                           'location': {'lat': 17.607789, 'lng': 8.081666},
                           'location_type': 'APPROXIMATE',
                           'viewport': {'northeast': {'lat': 23.4999997,
                                                      'lng': 15.9990339},
                                        'southwest': {'lat': 11.693756,
                                                      'lng': 0.1617177}}},
              'place_id': 'ChIJXa-hxnrsmRER1CSOFQ

{'results': [{'address_components': [{'long_name': 'Monaco',
                                      'short_name': 'MC',
                                      'types': ['country', 'political']}],
              'formatted_address': 'Monaco',
              'geometry': {'bounds': {'northeast': {'lat': 43.7519029,
                                                    'lng': 7.4426},
                                      'southwest': {'lat': 43.7237999,
                                                    'lng': 7.4091049}},
                           'location': {'lat': 43.73841760000001,
                                        'lng': 7.424615799999999},
                           'location_type': 'APPROXIMATE',
                           'viewport': {'northeast': {'lat': 43.7519029,
                                                      'lng': 7.4426},
                                        'southwest': {'lat': 43.7237999,
                                                      'lng': 7.4091049}

{'results': [{'address_components': [{'long_name': 'Tanzania',
                                      'short_name': 'TZ',
                                      'types': ['country', 'political']}],
              'formatted_address': 'Tanzania',
              'geometry': {'bounds': {'northeast': {'lat': -0.9843968000000001,
                                                    'lng': 40.6398},
                                      'southwest': {'lat': -11.7612539,
                                                    'lng': 29.34}},
                           'location': {'lat': -6.369028, 'lng': 34.888822},
                           'location_type': 'APPROXIMATE',
                           'viewport': {'northeast': {'lat': -0.9843968000000001,
                                                      'lng': 40.6398},
                                        'southwest': {'lat': -11.7612539,
                                                      'lng': 29.34}}},
              'place_id': 'ChIJEaF

{'results': [{'address_components': [{'long_name': 'Federated States of '
                                                   'Micronesia',
                                      'short_name': 'FM',
                                      'types': ['country', 'political']}],
              'formatted_address': 'Federated States of Micronesia',
              'geometry': {'bounds': {'northeast': {'lat': 10.2770863,
                                                    'lng': 163.5177612},
                                      'southwest': {'lat': 0.1538084,
                                                    'lng': 136.9226075}},
                           'location': {'lat': 7.425554, 'lng': 150.550812},
                           'location_type': 'APPROXIMATE',
                           'viewport': {'northeast': {'lat': 10.2770863,
                                                      'lng': 163.5177612},
                                        'southwest': {'lat': 0.1538084,
               

In [ ]:
# test = clean_covid.head(10)
# test


In [ ]:
# # Retriving lat and lng for locations
# for index,row in test.iterrows():
#     location = row["location"]
#     # Build the endpoint URL
#     target_url = f"https://maps.googleapis.com/maps/api/geocode/json?address={location}&key={g_key}"


#     # Run a request to endpoint and convert result to json
#     geo_data = requests.get(target_url).json()
#     # Print the json
#     pprint(geo_data)
#     try:
#         # Extract latitude and longitude for the locations in dataset
#         lat = geo_data["results"][0]["geometry"]["location"]["lat"]
#         test.loc[index,"Lat"] = lat
#         lng = geo_data["results"][0]["geometry"]["location"]["lng"]
#         test.loc[index,"Lng"] = lng
#     except:
#         print("Cannot find the location")

In [29]:
# Store latitude and longitude in locations, store total cases data in covid_total_cases
locations = clean_covid[["Lat", "Lng"]]
covid_total_cases = clean_covid["total_cases"]
covid_total_cases.head(10)

0    41221266.0
1    33289579.0
2    21006424.0
3     8057274.0
4     7290168.0
5     7055296.0
6     6992980.0
7     6682834.0
8     5318327.0
9     5226831.0
Name: total_cases, dtype: float64

In [31]:
gmaps.configure(api_key=g_key)
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
# The highest humidity for this dataframe is 100, set max_intensity to 100
heat_layer = gmaps.heatmap_layer(locations, weights=covid_total_cases, 
                                 dissipating=False, max_intensity=clean_covid["total_cases"].max()/8,
                                 point_radius=10)
# Add layer
fig.add_layer(heat_layer)
# markers = gmaps.marker_layer(locations)
# fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))